In [4]:
# 51
from functools import cache
from tqdm import tqdm

primes = [2]
for i in tqdm(range(3, 1000000)):
    for j in primes:
        if i % j == 0:
            break
        if j > i**0.5:
            primes.append(i)
            break

str_primes = [str(p) for p in primes if 1000000 > p > 100000]


class PrefixTree:
    def __init__(self):
        self.childs: dict = {str(i): None for i in range(10)}
        self.is_leaf = False
        self.string = None

    def build(self, strings, order=0):
        if len(strings) == 1:
            self.is_leaf = True
            self.string = strings[0]
            return self
        else:
            by_number = {str(i): [] for i in range(10)}
            for string in strings:
                by_number[string[order]].append(string)
            for key in by_number:
                if by_number[key]:
                    self.childs[key] = PrefixTree().build(
                        by_number[key], order+1)
        return self

    @cache
    def search(self, pattern: str):
        rez = []
        if self.is_leaf:
            if self.string[-len(pattern):] == pattern or len(pattern) == 0:
                return [self.string]
            else:
                return []
        if len(pattern) == 0:
            return []
        if pattern[0] == "x":
            for sub_tree_key in self.childs:
                if not self.childs[sub_tree_key] is None:
                    rez += (self.childs[sub_tree_key].search(
                        pattern[1:].replace("x", sub_tree_key)))
        else:
            if not self.childs[pattern[0]] is None:
                rez += (self.childs[pattern[0]].search(pattern[1:]))
        return rez


t = PrefixTree().build(str_primes)
max_seq = []
for i in str_primes:
    for n in range(5):
        for m in range(5):
            for k in range(5):
                if n > m > k:
                    s_str = i[:n] + 'x' + i[n+1:]
                    s_str = s_str[:m] + 'x' + s_str[m+1:]
                    s_str = s_str[:k] + 'x' + s_str[k+1:]
                    rez = t.search(s_str)
                    if len(rez) > len(max_seq):
                        max_seq = rez
print(max_seq[0])

100%|██████████| 999997/999997 [00:03<00:00, 294020.18it/s]


121313


In [5]:
# 52
for i in range(1, 200000):
    nums = []
    for j in range(2, 7):
        nums.append(set(str(i*j)))
    if all([i == nums[0] for i in nums]):
        print(i)

142857


In [ ]:
# 53

from math import factorial as f
s = 0
for i in range(10, 101):
    for j in range(0, i):
        if f(i)/f(j)/f(i-j) > 10**6:
            s += 1
s

4075

In [6]:
# 54

from collections import Counter
import bisect


class Card:
    NOT_NUMERIC_RANKS = {'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}

    def __init__(self, card):
        self.suit = card[1]
        self.rank = int(
            card[0]) if card[0] in '23456789' else Card.NOT_NUMERIC_RANKS[card[0]]


class PokerHand:

    def __init__(self, hand):
        self.cards = sorted([Card(card) for card in hand], key=lambda x: x.rank)
        self.cards_ranks_count = dict()
        for i, j in Counter(card.rank for card in self.cards).items():
            if self.cards_ranks_count.get(j):
                bisect.insort(
                    self.cards_ranks_count[j], i, key=lambda x: -1 * x)
            else:
                self.cards_ranks_count[j] = [i]
        self.combination = None
        self.combination_cards = list()
        self.other_cards = list()
        self._check_combination()

    def _check_combination(self):
        def isStraighFlush():
            if isFlush() and isStraight():
                return 8

        def isFour():
            if self.cards_ranks_count.get(4):
                self.combination_cards = self.cards_ranks_count[4]
                self.other_cards = self.cards_ranks_count[1]
                return 7

        def isFullHouse():
            if self.cards_ranks_count.get(3) and self.cards_ranks_count.get(2):
                self.combination_cards = self.cards_ranks_count[3] + \
                    self.cards_ranks_count[2]
                return 6

        def isFlush():
            if len(set(card.suit for card in self.cards)) == 1:
                self.combination_cards = self.cards_ranks_count[1]
                return 5

        def isStraight():
            if (len(self.cards_ranks_count.get(1, [])) == 5 and all(self.cards[i+1].rank - self.cards[i].rank == 1 for i in range(4)) or
                    self.cards_ranks_count.get(1) == [2, 3, 4, 5, 14]):
                self.combination_cards = self.cards_ranks_count[1]
                return 4

        def isThree():
            if self.cards_ranks_count.get(3):
                self.combination_cards = self.cards_ranks_count[3]
                self.other_cards = self.cards_ranks_count[1]
                return 3

        def isTwoTwo():
            if len(self.cards_ranks_count.get(2, [])) == 2:
                self.combination_cards = self.cards_ranks_count[2]
                self.other_cards = self.cards_ranks_count[1]
                return 2

        def isTwo():
            if self.cards_ranks_count.get(2):
                self.combination_cards = self.cards_ranks_count[2]
                self.other_cards = self.cards_ranks_count[1]
                return 1

        def isHigh():
            self.other_cards = self.cards_ranks_count[1]
            return 0

        self.combination = isStraighFlush() or \
            isFour() or \
            isFullHouse() or \
            isFlush() or \
            isStraight() or \
            isThree() or \
            isTwoTwo() or \
            isTwo() or \
            isHigh()

    def compare(self, other: 'PokerHand'):
        if (self.combination, self.combination_cards, self.other_cards) > (other.combination, other.combination_cards, other.other_cards):
            return 1
        return 0


all_hands = open('files/0054_poker.txt', 'r').read().split('\n')
all_hands = [i.split() for i in all_hands]
all_hands = [(i[0:5], i[5: 10]) for i in all_hands]

win = 0

for i, j in all_hands:
    win += PokerHand(i).compare(PokerHand(j))
print(win)

376


In [6]:
# 55
max = 0
for i in range(10, 100):
    for j in range(10, 100):
        csum = sum(map(int, str(i**j)))
        if csum > max:
            max = csum
max

972

In [ ]:
# 56
def euclid(a, b):
    while b:
        a, b = b, a % b
    return a


pairs = []
x = (1, 2)
n = 0

for i in range(1, 1000):
    to_add = (x[0]+x[1], x[1])
    denomenator = euclid(to_add[0], to_add[1])
    to_add = (to_add[0]//denomenator, to_add[1]//denomenator)

    pairs.append(to_add)
    if len(str(pairs[-1][0])) > len(str(pairs[-1][1])):
        n += 1
    x = (x[1], x[0]+2*x[1])
n

153

In [ ]:
# 57
from numpy import array
from tqdm import tqdm
diag_nums = array([3, 5, 7])
add_nums = array([2, 4, 6])
primes = [2]
for i in tqdm(range(3, 1000000)):
    for j in primes:
        if i % j == 0:
            break
        if j > i**0.5:
            primes.append(i)
            break


def is_prime(n):
    sq = n**0.5
    for i in primes:
        if n % i == 0:
            return False
        if i > sq:
            return True
    return True


i = 3
primes_num = 3
x = 1
l = 4
while x > 0.1:
    i += 2
    add_nums += 8
    diag_nums = diag_nums + add_nums
    primes_num += sum(map(is_prime, diag_nums))
    l += 4
    x = primes_num/(l + 1)
i

100%|██████████| 999997/999997 [00:03<00:00, 283033.26it/s]


26241

In [ ]:
# 59
from collections import Counter
from string import ascii_lowercase, ascii_uppercase

all_letters = ascii_lowercase + ascii_uppercase


with open('files/0059_cipher.txt') as c:
    cipher = list(map(lambda x: int(x), c.read().split(',')))

key_len = 3
key = ""

cipher_parts = [cipher[i::key_len] for i in range(key_len)]


for key_num, cipher_part in enumerate(cipher_parts):
    excluded = []
    for cipher_letter in cipher_part:
        for letter in ascii_lowercase:
            if letter in excluded or cipher_letter > 32:
                continue
            if chr(cipher_letter ^ ord(letter)) not in all_letters:
                excluded.append(letter)

    not_excluded = []
    for letter in ascii_lowercase:
        if letter not in excluded:
            not_excluded.append(letter)

    assert len(not_excluded) == 1

    key += not_excluded[0]

rez = 0
for index, letter in enumerate(cipher):
    print(chr(letter ^ ord(key[index % key_len])), end="")
    rez += letter ^ ord(key[index % key_len])

rez

An extract taken from the introduction of one of Euler's most celebrated papers, "De summis serierum reciprocarum" [On the sums of series of reciprocals]: I have recently found, quite unexpectedly, an elegant expression for the entire sum of this series 1 + 1/4 + 1/9 + 1/16 + etc., which depends on the quadrature of the circle, so that if the true sum of this series is obtained, from it at once the quadrature of the circle follows. Namely, I have found that the sum of this series is a sixth part of the square of the perimeter of the circle whose diameter is 1; or by putting the sum of this series equal to s, it has the ratio sqrt(6) multiplied by s to 1 of the perimeter to the diameter. I will soon show that the sum of this series to be approximately 1.644934066842264364; and from multiplying this number by six, and then taking the square root, the number 3.141592653589793238 is indeed produced, which expresses the perimeter of a circle whose diameter is 1. Following again the same ste

129448

In [ ]:
# 60

# slow - need to improve with clique search
from tqdm import tqdm

primes = open('files/primes10^8.txt', 'r').read().split('\n')[:-1]
primes = list(map(int, primes))
primes_less_10000 = [prime for prime in primes if 2 < prime < 10000]
min = primes.index(113)
primes = set(primes)

concatenated_primes = {prime: {prime} for prime in primes_less_10000}

for p1 in tqdm(primes_less_10000):
    for p2 in primes_less_10000:
        if p1 != p2:
            if int(str(p1) + str(p2)) in primes and int(str(p2) + str(p1)) in primes:
                concatenated_primes[p1].add(p2)
                concatenated_primes[p2].add(p1)


concatenated_primes_with_5_concats = {}
for concatenated1 in tqdm(concatenated_primes):
    t = 0
    s1 = concatenated_primes[concatenated1]
    for concatenated2 in concatenated_primes:
        if concatenated1 != concatenated2:
            if len(concatenated_primes[concatenated1].intersection(concatenated_primes[concatenated2])) >= 5:
                t += 1
    if t >= 4:
        concatenated_primes_with_5_concats[concatenated1] = concatenated_primes[concatenated1]

keys = list(concatenated_primes_with_5_concats.keys())
sets = list(concatenated_primes_with_5_concats.values())

combs = [{i} for i in range(len(sets))]

for i in range(1, 5):
    for comb in combs:
        if len(comb) == i:
            for ind, pset in enumerate(sets):
                if ind not in comb:
                    presolve = set.intersection(*[sets[j] for j in comb])
                    if keys[ind] not in presolve:
                        continue
                    if len(set.intersection(pset, presolve)) > 4:
                        if {*comb, ind} not in combs:
                            combs.append({*comb, ind})
    combs = [comb for comb in combs if len(comb) > i]


combs = [[keys[i] for i in comb] for comb in combs]

sum(combs[0])

100%|██████████| 1228/1228 [00:02<00:00, 610.41it/s]


26033

In [45]:
# 61
from itertools import permutations
triangle = [0]
i = 1
while (t_i := i*(i+1)//2) < 10000:
    triangle.append(t_i)
    i += 1

polygonal_nums = [[(str(num + triangle[ind] * fig)) for ind, num in enumerate(triangle[1:])
                   if 1000 < num + triangle[ind] * fig < 10000] for fig in range(6)]

def find_next_num(nums, poly_nums):
    if len(poly_nums) == 0:
        if nums[-1][2:] == nums[0][:2]:
            print(sum(map(int, nums)))
            raise
        return
    for poly_num in poly_nums[0]:
        if nums[-1][2:] == poly_num[:2]:
            find_next_num(nums + [poly_num], poly_nums[1:])

try:
    for perm in permutations(polygonal_nums):
        for t_i in perm[0]:
            find_next_num([t_i], perm[1:])
except:
    pass

28684
